In [1]:
import geopandas as gpd
import json 

In [2]:
# Constants, will come from inputRWHALE.json
Constants = json.loads(
    ''' {
        "START_TIME_STEP": 0,
        "MAX_TIME_STEP": 3650,
        "DISASTER_TIME_STEP": 1,
        "DS4_REPAIR_DURATION": 240,
        "MAX_REPAIR_CREW_DEMAND_PER_BUILDING": 50,
        "MAX_RESIDENTS_PER_BUILDING": 2000,
        "HOUSING_RESOURCES": ["Shelter"],
        "REPAIR_CREW_DEMAND_PER_SQFT": {
            "DS1": 5400,
            "DS2": 5400,
            "DS3": 2700,
            "DS4": 2700
        },
        "DEFAULT_REPAIR_DURATION_DICT": {
            "Lognormal": {
                "Median": 0,
                "Dispersion": 0.3
            }
        }
    }'''
)
Constants

{'START_TIME_STEP': 0,
 'MAX_TIME_STEP': 3650,
 'DISASTER_TIME_STEP': 1,
 'DS4_REPAIR_DURATION': 240,
 'MAX_REPAIR_CREW_DEMAND_PER_BUILDING': 50,
 'MAX_RESIDENTS_PER_BUILDING': 2000,
 'HOUSING_RESOURCES': ['Shelter'],
 'REPAIR_CREW_DEMAND_PER_SQFT': {'DS1': 5400,
  'DS2': 5400,
  'DS3': 2700,
  'DS4': 2700},
 'DEFAULT_REPAIR_DURATION_DICT': {'Lognormal': {'Median': 0,
   'Dispersion': 0.3}}}

In [3]:
localityBoundaryFile = "../Census_Tract_Boundaries_AI.geojson"
boundaryFile = localityBoundaryFile
boundaryGDF = gpd.read_file(boundaryFile)
boundaryGDF.head(2)

,OBJECTID,DIST_NAME,DISTRICT_ID,SHAPE_Length,SHAPE_Area,geometry
0,122,CENSUS TRACT #4271,327,0.050782,0.000119,"MULTIPOLYGON (((-122.22365 37.75693, -122.2235..."
1,123,CENSUS TRACT #4272,328,0.081200,0.000207,"MULTIPOLYGON (((-122.24483 37.77906, -122.2435..."


In [6]:
boundaryGDF.loc[0,"geometry"].centroid.xy

(array('d', [-122.23103896598545]), array('d', [37.764663304632116]))

In [10]:
# Define contents
localityBoundaryFile = "../Census_Tract_Boundaries_AI.geojson"
componentsFile = "./Example 5/tmp.SimCenter/Results/"
boundaryFile = localityBoundaryFile
boundaryGDF = gpd.read_file(boundaryFile)
Content = dict()
for row_ind in boundaryGDF.index:
    locality_i = dict()
    components = dict()
    geom_i = boundaryGDF.loc[row_ind, "geometry"]
    loc_id = str(row_ind)
    components.update({"Geometry": geom_i.wkt})
    components.update({"RecoveryResourceSuppliers":["EmergencyResponseCenter"]})
    components.update({"ComponentsInfoFolder":componentsFile})
    components.update({"AreaPerPerson":541})
    locality_i.update({"ComponentsInLocality":components})
    Content.update({loc_id: locality_i})


In [12]:
# Define DamageInput come from inputRWHALE and the scenarioID should be a list
# a sample of scenarios are selected from DL realizations to estimate the mean
# and std of the recovery process. Assuming normal, 20-30 realizations may be needed
DamageInput = json.loads(
    '''{"Type": "R2DDamageInput",
        "Parameters": {
            "ScenarioID": [1]
        }}'''
)
DamageInput

{'Type': 'R2DDamageInput', 'Parameters': {'ScenarioID': [1]}}

In [14]:
# Define DistributionModels and DistributionPriorities
# This list are built in and users can choose from
# Currently only have UtilityDistributionModel. Maybe have ResidualDemandModel
# in the future
# Created a DistributionPriorities to define default priorities
DistributionModelTypes = dict({"UtilityDistributionModel":"UtilityDistributionModel"})
DistributionPriorities = dict()
SupplierOnlyDistributionPriorityPara = json.loads(
    '''{
    }'''
)
RandomPriorityPara = json.loads(
    '''{
        "Seed": 42.0,
        "DemandType": [
            "RecoveryDemand"
        ]
        }'''
)
DistributionPriorities.update({
    "SupplierOnlyDistributionPriority":SupplierOnlyDistributionPriorityPara,
    "RandomPriority":RandomPriorityPara
})
DistributionPriorities


{'SupplierOnlyDistributionPriority': {},
 'RandomPriority': {'Seed': 42.0, 'DemandType': ['RecoveryDemand']}}

In [52]:
# Create Resources. Will come from inputRWHALE.json
Resources = json.loads(
    '''{
    "Shelter": {
            "Group": "Utilities",
            "DistributionModel": {
                "Type":"UtilityDistributionModel",
                "Parameters":{
                    "DistributionPriority": {
                        "Type": "SupplierOnlyDistributionPriority"
                    }
                }
            }
        },
    "FirstResponderEngineer": {
            "Group": "RecoveryResources",
            "DistributionModel": {
                "Type": "UtilityDistributionModel",
                "Parameters": {
                    "DistributionPriority": {
                        "Type": "RandomPriority"
                    }
                }
            }
        },
     "SeniorEngineer": {
            "Group": "RecoveryResources",
            "DistributionModel": {
                "Type": "UtilityDistributionModel",
                "Parameters": {
                    "DistributionPriority": {
                        "Type": "RandomPriority"
                    }
                }
            }
        },
    "Contractor": {
            "Group": "RecoveryResources",
            "DistributionModel": {
                "Type": "UtilityDistributionModel",
                "Parameters": {
                    "DistributionPriority": {
                        "Type": "RandomPriority"
                    }
                }
            }
        },
    "Money": {
            "Group": "RecoveryResources",
            "DistributionModel": {
                "Type": "UtilityDistributionModel",
                "Parameters": {
                    "DistributionPriority": {
                        "Type": "RandomPriority"
                    }
                }
            }
        },
    "PlanCheckEngineeringTeam": {
            "Group": "RecoveryResources",
            "DistributionModel": {
                "Type": "UtilityDistributionModel",
                "Parameters": {
                    "DistributionPriority": {
                        "Type": "RandomPriority"
                    }
                }
            }
        },
    "SitePreparationCrew": {
            "Group": "RecoveryResources",
            "DistributionModel": {
                "Type": "UtilityDistributionModel",
                "Parameters": {
                    "DistributionPriority": {
                        "Type": "RandomPriority"
                    }
                }
            }
        },
    "CleanUpCrew": {
            "Group": "RecoveryResources",
            "DistributionModel": {
                "Type": "UtilityDistributionModel",
                "Parameters": {
                    "DistributionPriority": {
                        "Type": "RandomPriority"
                    }
                }
            }
        },
    "EngineeringDesignTeam": {
            "Group": "RecoveryResources",
            "DistributionModel": {
                "Type": "UtilityDistributionModel",
                "Parameters": {
                    "DistributionPriority": {
                        "Type": "RandomPriority"
                    }
                }
            }
        },
    "DemolitionCrew": {
            "Group": "RecoveryResources",
            "DistributionModel": {
                "Type": "UtilityDistributionModel",
                "Parameters": {
                    "DistributionPriority": {
                        "Type": "RandomPriority"
                    }
                }
            }
        },
    "RepairCrew": {
            "Group": "RecoveryResources",
            "DistributionModel": {
                "Type": "UtilityDistributionModel",
                "Parameters": {
                    "DistributionPriority": {
                        "Type": "RandomPriority"
                    }
                }
            }
        }
    }'''
)



In [64]:
# Define ResilienceCalculator should come from inputRWHALE
ResilienceCalculator = json.loads(
    '''[
        {"Type": "ReCoDeSResilienceCalculator",
            "Parameters": {
                "Scope": "All",
                "Resources": [
                    "Shelter",
                    "FirstResponderEngineer",
                    "RepairCrew",
                    "Money"
                ]
            }
        },
        {
            "Type": "NISTGoalsResilienceCalculator",
            "Parameters": [
                {
                    "Resource": "Shelter",
                    "Scope": "All",
                    "DesiredFunctionalityLevel": 0.95
                }
            ]
        }
    ]
    '''
)
ResilienceCalculator


[{'Type': 'ReCoDeSResilienceCalculator',
  'Parameters': {'Scope': 'All',
   'Resources': ['Shelter', 'FirstResponderEngineer', 'RepairCrew', 'Money']}},
 {'Type': 'NISTGoalsResilienceCalculator',
  'Parameters': [{'Resource': 'Shelter',
    'Scope': 'All',
    'DesiredFunctionalityLevel': 0.95}]}]